In [10]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform,data
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform,data
%matplotlib inline
import sys,os,caffe
import cv2
import math
import time
from skimage.transform import rotate
from numpy.lib.stride_tricks import as_strided as ast

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# RGB -> YCbCr
def rgb2ycbcr(rgb):
    m = np.array([[ 65.481, 128.553, 24.966],
                  [-37.797, -74.203, 112],
                  [ 112, -93.786, -18.214]])
    shape = rgb.shape
    if len(shape) == 3:
        rgb = rgb.reshape((shape[0] * shape[1], 3))
    ycbcr = np.dot(rgb, m.transpose() / 255.)
    ycbcr[:,0] += 16.
    ycbcr[:,1:] += 128.
    return ycbcr.reshape(shape)

def psnr_RGB(img1, img2, boundary):
    HR = img1[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary,:]
    LR = img2[boundary:img2.shape[0]-boundary,boundary:img2.shape[1]-boundary,:]
    mse = np.mean( (HR - LR) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def psnr_Y(img1, img2, boundary):
    HR = img1[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary]
    LR = img2[boundary:img2.shape[0]-boundary,boundary:img2.shape[1]-boundary]
    mse = np.mean( (HR - LR) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def ssim(img1, img2, boundary):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    HR = img1[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary].astype(np.float64)
    LR = img2[boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary].astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(HR, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(LR, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(HR**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(LR**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(HR * LR, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
     # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
     
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
     
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
     
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))
    
caffe_root = '/workspace/examples/caffe_test' 
#sys.path.insert(0, 'examples/caffe_test')
os.chdir(caffe_root)

GPU_ID = 1
caffe.set_mode_gpu()
caffe.set_device(GPU_ID)


model_def = 'VAE/VAE_Deploy.prototxt'


'''
for k in range(155000,5000,-5000):
    Model_root = 'VAE/Model/Train_iter_'+str(k)+'.caffemodel'

    model_weights =  Model_root

    net = caffe.Net(model_def, model_weights, caffe.TEST)
    print "caffemodel ", k
    
    psnr_ave = 0
    up_scale = 1
    patch_size_x = 128
    patch_size_y = 128
    stride_x = 64
    stride_y = 64
    channel_swap1 = (2, 0, 1)
    channel_swap2 = (1, 2, 0)
    start = time.time()
    for file in os.listdir("NTIRE/test/HR/"):
        if file.endswith(".png"):
            write_name = file
            HR = cv2.imread("NTIRE/test/HR/" + write_name)
            HR_float = np.float32(HR[...,::-1])
            LR = cv2.imread("NTIRE/test/Bic/" + write_name)
            LR_float = np.float32(LR[...,::-1])/255
            shape_ori = HR.shape
            SR_final = np.zeros((shape_ori[0],shape_ori[1],3)).astype(np.float32)
            for d in range(0,4,1):

                LR_rotate = np.rot90(LR_float,d)
    
                shape = LR_rotate.shape
        

                mask = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                SR = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
    
                num_x = round(float(shape[0]) / float(stride_x) + 0.4)
                num_y = round(float(shape[1]) / float(stride_y) + 0.4)

                for i in range(0, shape[0], stride_x):
                    for j in range(0, shape[1], stride_y):
                        mask_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                        SR_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                        x = min(shape[0], i+patch_size_x)
                        y = min(shape[1], j+patch_size_y)

                        A = LR_rotate[x-patch_size_x:x,y-patch_size_y:y,:]
                        #print A.shape

                        net.blobs['data'].data[...] = A.transpose(channel_swap1)
            
                        net.forward()
                        patch_SR = net.blobs['SR'].data[0,:,:,:].transpose(channel_swap2)
                
                        #print k, d, x, y
                        mask_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = 1.0
                        SR_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = patch_SR
                        mask = mask + mask_patch
                        SR = SR + SR_patch

                SR = SR/mask
                #print SR.shape
                SR_final = SR_final + np.rot90(SR,4-d)*0.25*255

                #cv2.imwrite(image_root + "SR_" + str(k+1) + ".png",SR_final[...,::-1])

            if sum(sum(HR_float[:,:,0]-HR_float[:,:,1])) == 0:
                HR_YUV = HR_float
                SR_YUV = SR_final
            else:
                HR_YUV = rgb2ycbcr(HR_float)
                SR_YUV = rgb2ycbcr(SR_final)
            HR_Y = HR_YUV[:,:,0]
            SR_Y = SR_YUV[:,:,0]
            value1 = psnr_Y(SR_Y, HR_Y, 4)
            #value2 = ssim(SR_Y, HR_Y, 4)
            psnr_ave = psnr_ave + value1/10
            #ssim_ave = ssim_ave + value2/10
    
    end = time.time()
    cost = end - start
    #print "time cost is ", cost
    print "average PSNR is ", psnr_ave
    
'''
Model_root = 'VAE/Model/Train_iter_105000.caffemodel'

model_weights = Model_root

net = caffe.Net(model_def, model_weights, caffe.TEST)
    
psnr_ave = 0
ssim_ave = 0
up_scale = 1
patch_size_x = 128
patch_size_y = 128
stride_x = 64
stride_y = 64
channel_swap1 = (2, 0, 1)
channel_swap2 = (1, 2, 0)
start = time.time()
for file in os.listdir("NTIRE/Set5/4x/HR/"):
    if file.endswith(".png"):
        write_name = file
        HR = cv2.imread("NTIRE/Set5/4x/HR/" + write_name)
        HR_float = np.float32(HR[...,::-1])
        LR = cv2.imread("NTIRE/Set5/4x/Bic/" + write_name)
        LR_float = np.float32(LR[...,::-1])/255
        shape_ori = HR.shape
        SR_final = np.zeros((shape_ori[0],shape_ori[1],3)).astype(np.float32)
        for d in range(0,4,1):

            LR_rotate = np.rot90(LR_float,d)
    
            shape = LR_rotate.shape
        

            mask = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
            SR = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
    
            num_x = round(float(shape[0]) / float(stride_x) + 0.4)
            num_y = round(float(shape[1]) / float(stride_y) + 0.4)

            for i in range(0, shape[0], stride_x):
                for j in range(0, shape[1], stride_y):
                    mask_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                    SR_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                    x = min(shape[0], i+patch_size_x)
                    y = min(shape[1], j+patch_size_y)

                    A = LR_rotate[x-patch_size_x:x,y-patch_size_y:y,:]
                    #print A.shape

                    net.blobs['data'].data[...] = A.transpose(channel_swap1)
            
                    net.forward()
                    patch_SR = net.blobs['SR'].data[0,:,:,:].transpose(channel_swap2)
                
                    #print k, d, x, y
                    mask_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = 1.0
                    SR_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = patch_SR
                    mask = mask + mask_patch
                    SR = SR + SR_patch

            SR = SR/mask
            #print SR.shape
            SR_final = SR_final + np.rot90(SR,4-d)*0.125*255
        
        for d in range(0,4,1):

            LR_rotate = np.rot90(np.fliplr(LR_float),d)
    
            shape = LR_rotate.shape
        

            mask = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
            SR = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
    
            num_x = round(float(shape[0]) / float(stride_x) + 0.4)
            num_y = round(float(shape[1]) / float(stride_y) + 0.4)

            for i in range(0, shape[0], stride_x):
                for j in range(0, shape[1], stride_y):
                    mask_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                    SR_patch = np.zeros((shape[0]*up_scale,shape[1]*up_scale,3)).astype(np.float32)
                    x = min(shape[0], i+patch_size_x)
                    y = min(shape[1], j+patch_size_y)

                    A = LR_rotate[x-patch_size_x:x,y-patch_size_y:y,:]
                    #print A.shape

                    net.blobs['data'].data[...] = A.transpose(channel_swap1)
            
                    net.forward()
                    patch_SR = net.blobs['SR'].data[0,:,:,:].transpose(channel_swap2)
                
                    #print k, d, x, y
                    mask_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = 1.0
                    SR_patch[(x-patch_size_x)*up_scale:x*up_scale, (y-patch_size_y)*up_scale:y*up_scale,:] = patch_SR
                    mask = mask + mask_patch
                    SR = SR + SR_patch

            SR = SR/mask
            #print SR.shape
            SR_final = SR_final + np.fliplr(np.rot90(SR,4-d))*0.125*255
        
        cv2.imwrite("NTIRE/Set5/4x/SR_VAE_P/" + write_name,SR_final[...,::-1])
    
        #HR_rotate = rotate_bound(HR_float,d)
        if sum(sum(HR_float[:,:,0]-HR_float[:,:,1])) == 0:
            HR_YUV = HR_float
            SR_YUV = SR_final
        else:
            HR_YUV = rgb2ycbcr(HR_float)
            SR_YUV = rgb2ycbcr(SR_final)
        HR_Y = HR_YUV[:,:,0]
        SR_Y = SR_YUV[:,:,0]
        value1 = psnr_Y(SR_Y, HR_Y, 4)
        value2 = ssim(SR_Y, HR_Y, 4)
        psnr_ave = psnr_ave + value1/5
        ssim_ave = ssim_ave + value2/5
    
end = time.time()
cost = end - start
print "time cost is ", cost
print "average PSNR is ", psnr_ave
print "average SSIM is ", ssim_ave




time cost is  6.55231785774
average PSNR is  31.5511806468
average SSIM is  0.8871480578240747
